# Boxplot on AssessmentScore data, overlay w dots
In this notebook, I experiment my way into

1. [Set up](https://www.codingforentrepreneurs.com/blog/use-django-in-jupyter)
   Jupyter for Django:
   - I copied in the file `django_for_jupyter.py` in project root directory.
   - Same location, now issue `DJANGO_PROJECT="box_whiskers_demo" jupyter lab`.
   - This will start Jupyter lab **in browser** - and thus 
     access to **this Notebook** with Django project access.
     Does logic seem backward? Sorry!
1. **Retrieve data** from model `AssessmentScore`:
1. Preprocess (if necessary) in order to **prepare for 12-box-plot**.
1. Plot and label
1. **Overlay** with arbitraty, individual student data (**dot** plot).

In [1]:
from django_for_jupyter import init_django
init_django()

In [2]:
from boxplot.models import Elev

Elev.objects.all()

<QuerySet [<Elev: Andersine Andersen, 1a (2019), Frederiksborg Gymnasium og HF>, <Elev: Sida Dida, 2p (2018), Frederiksborg Gymnasium og HF>, <Elev: Camilla Horserød, 1test (2020), Prøvens Gymnasium>, <Elev: Camilla Vejlebyskov, 1test (2020), Prøvens Gymnasium>, <Elev: Emma Oue, 1test (2020), Prøvens Gymnasium>, <Elev: Helle Barup, 1test (2020), Prøvens Gymnasium>, <Elev: Helle Byskov, 1test (2020), Prøvens Gymnasium>, <Elev: Henrik Menstrup, 1test (2020), Prøvens Gymnasium>, <Elev: Henrik Voer Hede, 1test (2020), Prøvens Gymnasium>, <Elev: Ida Himmark Mark, 1test (2020), Prøvens Gymnasium>, <Elev: Jens Febbersted, 1test (2020), Prøvens Gymnasium>, <Elev: Jens Molshuse, 1test (2020), Prøvens Gymnasium>, <Elev: Jørgen Hallenslev Gårde, 1test (2020), Prøvens Gymnasium>, <Elev: Jørgen Normark, 1test (2020), Prøvens Gymnasium>, <Elev: Jørgen Storå, 1test (2020), Prøvens Gymnasium>, <Elev: Jørgen Trængstrup, 1test (2020), Prøvens Gymnasium>, <Elev: Kirsten Dybdal, 1test (2020), Prøvens Gymn